# 🎯 Attention Mechanism Tutorial

이 노트북에서는 Attention 메커니즘을 처음부터 구현하고 실험합니다.
"Attention is All You Need" 논문의 핵심 개념을 직접 코딩해봅니다.

## 1. 환경 설정

In [ ]:
import numpy as np
import sys
import os

# 상위 디렉토리를 path에 추가
sys.path.append(os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath('.')))))

# core 모듈 import
from core.attention import (
    scaled_dot_product_attention,
    MultiHeadAttention,
    positional_encoding,
    add_positional_encoding,
    create_causal_mask,
    visualize_attention
)

print("✅ 환경 설정 완료!")
print(f"NumPy 버전: {np.__version__}")

## 2. Attention의 직관적 이해

In [ ]:
# Attention의 핵심: Query, Key, Value
print("🔍 Attention의 핵심 개념")
print("=" * 50)
print()
print("📝 비유: 도서관에서 책 찾기")
print("  Query (질문): '파이썬 프로그래밍 책을 찾고 있어요'")
print("  Key (색인): 각 책의 제목과 주제")
print("  Value (내용): 실제 책의 내용")
print()
print("→ Attention은 Query와 가장 관련있는 Key를 찾아")
print("  해당하는 Value를 가져오는 메커니즘입니다.")
print()
print("수식: Attention(Q,K,V) = softmax(QK^T/√d_k)V")

## 3. Scaled Dot-Product Attention 구현

In [ ]:
# 간단한 예제로 시작
print("📊 Simple Attention Example")
print("=" * 50)

# 3개의 단어, 각 4차원 벡터
seq_len = 3
d_model = 4

# Query, Key, Value 생성
np.random.seed(42)
Q = np.random.randn(seq_len, d_model)
K = np.random.randn(seq_len, d_model)
V = np.random.randn(seq_len, d_model)

print(f"Q shape: {Q.shape}")
print(f"K shape: {K.shape}")
print(f"V shape: {V.shape}")

# Attention 계산
output, attention_weights = scaled_dot_product_attention(Q, K, V)

print(f"\n출력 shape: {output.shape}")
print(f"Attention weights shape: {attention_weights.shape}")
print(f"\nAttention weights:\n{attention_weights}")
print(f"\n각 행의 합 (should be 1): {attention_weights.sum(axis=1)}")

In [ ]:
# Attention 계산 단계별 분해
print("🔧 Attention 계산 단계별 분해")
print("=" * 50)

# Step 1: Q와 K의 내적
scores = Q @ K.T
print("Step 1 - Scores (QK^T):")
print(scores)
print(f"Shape: {scores.shape}\n")

# Step 2: Scaling
d_k = K.shape[-1]
scaled_scores = scores / np.sqrt(d_k)
print(f"Step 2 - Scaled scores (÷√{d_k}):")
print(scaled_scores)
print()

# Step 3: Softmax
def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

attention_weights_manual = softmax(scaled_scores)
print("Step 3 - Attention weights (softmax):")
print(attention_weights_manual)
print()

# Step 4: 가중합
output_manual = attention_weights_manual @ V
print("Step 4 - Output (weighted sum of V):")
print(output_manual)

# 검증
print(f"\n✅ 수동 계산과 함수 결과 동일: {np.allclose(output, output_manual)}")

## 4. Self-Attention 실습

In [ ]:
# 문장에서 Self-Attention
print("📝 문장에서 Self-Attention")
print("=" * 50)

# 간단한 문장
sentence = "The cat sat"
tokens = sentence.split()
print(f"문장: '{sentence}'")
print(f"토큰: {tokens}\n")

# 각 단어를 임의의 벡터로 표현
np.random.seed(42)
word_embeddings = {
    "The": np.random.randn(8),
    "cat": np.random.randn(8),
    "sat": np.random.randn(8)
}

# 임베딩 행렬 생성
X = np.array([word_embeddings[token] for token in tokens])
print(f"임베딩 shape: {X.shape}")

# Self-attention (Q=K=V=X)
output, attention_weights = scaled_dot_product_attention(X, X, X)

# Attention 시각화
print("\nAttention Matrix:")
print("       ", end="")
for token in tokens:
    print(f"{token:>8}", end="")
print()

for i, token in enumerate(tokens):
    print(f"{token:>7}", end="")
    for j in range(len(tokens)):
        weight = attention_weights[i, j]
        print(f"{weight:8.3f}", end="")
    print()

# 해석
print("\n💡 해석:")
for i, token in enumerate(tokens):
    max_idx = np.argmax(attention_weights[i])
    print(f"  '{token}'이 가장 주목하는 단어: '{tokens[max_idx]}' "
          f"(weight: {attention_weights[i, max_idx]:.3f})")

## 5. Causal Attention (GPT 스타일)

In [ ]:
# Causal mask 생성 및 적용
print("🔮 Causal Attention (미래를 볼 수 없음)")
print("=" * 50)

# 더 긴 문장
sentence = "I think therefore I am"
tokens = sentence.split()
seq_len = len(tokens)
print(f"문장: '{sentence}'")
print(f"토큰: {tokens}\n")

# 임베딩
np.random.seed(42)
X = np.random.randn(seq_len, 16)

# Causal mask 생성
causal_mask = create_causal_mask(seq_len)
print("Causal Mask:")
print(causal_mask)
print()

# Causal attention 적용
output, attention_weights = scaled_dot_product_attention(X, X, X, mask=causal_mask)

# 시각화
print("Causal Attention Weights:")
print("       ", end="")
for token in tokens:
    print(f"{token:>10}", end="")
print()

for i, token in enumerate(tokens):
    print(f"{token:>7}", end="")
    for j in range(seq_len):
        weight = attention_weights[i, j]
        if weight < 0.001:
            print("         -", end="")
        else:
            print(f"{weight:10.3f}", end="")
    print()

print("\n💡 특징:")
print("  - 하삼각 행렬 형태")
print("  - 각 토큰은 자신과 이전 토큰만 볼 수 있음")
print("  - GPT와 같은 생성 모델에서 사용")

## 6. Multi-Head Attention

In [ ]:
# Multi-Head Attention 실습
print("🎭 Multi-Head Attention")
print("=" * 50)

# 설정
d_model = 64
num_heads = 8
seq_len = 10

print(f"Model dimension: {d_model}")
print(f"Number of heads: {num_heads}")
print(f"Dimension per head: {d_model // num_heads}\n")

# Multi-Head Attention 생성
mha = MultiHeadAttention(d_model, num_heads)

# 입력 데이터
X = np.random.randn(seq_len, d_model)
print(f"입력 shape: {X.shape}")

# Forward pass
output, attention_weights = mha.forward(X, X, X)

print(f"출력 shape: {output.shape}")
print(f"Attention weights shape: {attention_weights.shape}")
print(f"  (num_heads, seq_len, seq_len)\n")

# 각 head의 attention 패턴 분석
print("각 Head의 특성:")
for head_idx in range(num_heads):
    head_weights = attention_weights[head_idx]
    
    # 대각선 강도 (자기 자신에 대한 attention)
    diagonal_strength = np.mean(np.diag(head_weights))
    
    # 분산 (attention의 집중도)
    variance = np.var(head_weights)
    
    print(f"  Head {head_idx + 1}: "
          f"대각선 강도={diagonal_strength:.3f}, "
          f"분산={variance:.3f}")

In [ ]:
# Multi-Head의 장점 시연
print("💡 Multi-Head의 장점")
print("=" * 50)
print()
print("Single Head vs Multi-Head 비교:")
print()

# Single Head (큰 차원)
single_head = MultiHeadAttention(d_model=64, num_heads=1)

# Multi-Head (여러 작은 차원)
multi_head = MultiHeadAttention(d_model=64, num_heads=8)

# 동일한 입력
X = np.random.randn(10, 64)

# Forward pass
output_single, weights_single = single_head.forward(X, X, X)
output_multi, weights_multi = multi_head.forward(X, X, X)

print("1. Single Head:")
print(f"   - 1개의 64차원 attention")
print(f"   - 하나의 관점만 학습")
print(f"   - Attention 분산: {np.var(weights_single):.4f}")
print()

print("2. Multi-Head (8 heads):")
print(f"   - 8개의 8차원 attention")
print(f"   - 다양한 관점 동시 학습")

# 각 head의 다양성
head_variances = [np.var(weights_multi[i]) for i in range(8)]
print(f"   - Head별 분산: {[f'{v:.4f}' for v in head_variances[:4]]} ...")
print(f"   - 평균 분산: {np.mean(head_variances):.4f}")
print()
print("→ Multi-Head는 다양한 패턴을 동시에 학습할 수 있습니다!")

## 7. Positional Encoding

In [ ]:
# Positional Encoding 생성 및 분석
print("📍 Positional Encoding")
print("=" * 50)

# PE 생성
seq_len = 50
d_model = 128

pe = positional_encoding(seq_len, d_model)
print(f"PE shape: {pe.shape}")
print(f"PE 값 범위: [{pe.min():.3f}, {pe.max():.3f}]\n")

# 처음 몇 개 위치의 패턴
print("처음 5개 위치의 처음 8차원:")
print("Pos  ", end="")
for dim in range(8):
    print(f"Dim{dim:2d}  ", end="")
print()

for pos in range(5):
    print(f"{pos:3d}  ", end="")
    for dim in range(8):
        val = pe[pos, dim]
        print(f"{val:6.3f} ", end="")
    print()

# 주파수 특성
print("\n📊 차원별 주파수 특성:")
for dim in [0, 32, 64, 127]:
    # 주기 계산
    if dim % 2 == 0:
        wavelength = 2 * np.pi * (10000 ** (dim / d_model))
        print(f"  Dim {dim:3d}: 파장 ≈ {wavelength:.1f} positions")

In [ ]:
# PE의 효과 시연
print("🔬 Positional Encoding의 효과")
print("=" * 50)

# 동일한 단어가 다른 위치에 있을 때
sentence = "The cat and the dog"
tokens = sentence.split()
print(f"문장: '{sentence}'")
print(f"토큰: {tokens}")
print(f"  'the'가 위치 0과 3에 나타남\n")

# 간단한 임베딩 (the는 같은 벡터)
embedding_dim = 32
word_embeddings = {
    "The": np.ones(embedding_dim) * 0.1,
    "the": np.ones(embedding_dim) * 0.1,  # 같은 벡터
    "cat": np.ones(embedding_dim) * 0.2,
    "and": np.ones(embedding_dim) * 0.3,
    "dog": np.ones(embedding_dim) * 0.4
}

# 임베딩 행렬
X = np.array([word_embeddings[token] for token in tokens])

# PE 없이
print("1. PE 없이:")
print(f"   'The' 벡터 평균: {X[0].mean():.3f}")
print(f"   'the' 벡터 평균: {X[3].mean():.3f}")
print(f"   → 동일한 벡터!\n")

# PE 추가
X_with_pe = add_positional_encoding(X)
print("2. PE 추가 후:")
print(f"   'The' (pos 0) 벡터 평균: {X_with_pe[0].mean():.3f}")
print(f"   'the' (pos 3) 벡터 평균: {X_with_pe[3].mean():.3f}")
print(f"   → 다른 벡터!")
print()
print("→ PE를 통해 위치 정보가 추가되어 같은 단어도 구별 가능")

## 8. 실전 예제: 간단한 Transformer Block

In [ ]:
# 간단한 Transformer Block 구현
class SimpleTransformerBlock:
    """간단한 Transformer 블록"""
    
    def __init__(self, d_model, num_heads, dropout_rate=0.1):
        self.attention = MultiHeadAttention(d_model, num_heads, dropout_rate)
        self.d_model = d_model
        
        # Feed-forward network weights
        self.ff_w1 = np.random.randn(d_model, d_model * 4) * 0.1
        self.ff_w2 = np.random.randn(d_model * 4, d_model) * 0.1
    
    def forward(self, x, mask=None):
        # 1. Multi-Head Attention
        attn_output, attn_weights = self.attention.forward(x, x, x, mask)
        
        # 2. Residual connection + Layer Norm (simplified)
        x = x + attn_output
        x = self.layer_norm(x)
        
        # 3. Feed-forward
        ff_output = self.feed_forward(x)
        
        # 4. Residual connection + Layer Norm
        x = x + ff_output
        x = self.layer_norm(x)
        
        return x, attn_weights
    
    def feed_forward(self, x):
        """Position-wise feed-forward network"""
        # Linear -> ReLU -> Linear
        hidden = np.maximum(0, x @ self.ff_w1)  # ReLU
        output = hidden @ self.ff_w2
        return output
    
    def layer_norm(self, x, eps=1e-6):
        """Layer normalization"""
        mean = x.mean(axis=-1, keepdims=True)
        std = x.std(axis=-1, keepdims=True)
        return (x - mean) / (std + eps)

# Transformer Block 테스트
print("🏗️ Simple Transformer Block")
print("=" * 50)

# 생성
transformer = SimpleTransformerBlock(d_model=64, num_heads=8)

# 입력 (with PE)
seq_len = 10
X = np.random.randn(seq_len, 64)
X = add_positional_encoding(X)

# Forward pass
output, attention = transformer.forward(X)

print(f"입력 shape: {X.shape}")
print(f"출력 shape: {output.shape}")
print(f"Attention shape: {attention.shape}")
print()
print("구성 요소:")
print("  1. Multi-Head Attention")
print("  2. Residual Connection")
print("  3. Layer Normalization")
print("  4. Feed-Forward Network")
print("  5. Another Residual + LayerNorm")
print()
print("→ 이것이 Transformer의 기본 블록입니다!")

## 9. Attention 패턴 실험

In [ ]:
# 다양한 Attention 패턴 생성 및 비교
print("🎨 다양한 Attention 패턴")
print("=" * 50)

seq_len = 8

# 1. Identity Attention (자기 자신만)
identity_attention = np.eye(seq_len)

# 2. Uniform Attention (균등)
uniform_attention = np.ones((seq_len, seq_len)) / seq_len

# 3. Local Attention (인접 토큰)
local_attention = np.zeros((seq_len, seq_len))
for i in range(seq_len):
    for j in range(max(0, i-1), min(seq_len, i+2)):
        local_attention[i, j] = 1/3

# 4. Global + Local (혼합)
mixed_attention = 0.7 * local_attention + 0.3 * uniform_attention

# 시각화
patterns = [
    ("Identity (자기 자신만)", identity_attention),
    ("Uniform (균등 분포)", uniform_attention),
    ("Local (인접 토큰)", local_attention),
    ("Mixed (Global + Local)", mixed_attention)
]

for name, pattern in patterns:
    print(f"\n{name}:")
    for i in range(min(5, seq_len)):
        print(" ", end="")
        for j in range(min(8, seq_len)):
            val = pattern[i, j]
            if val > 0.5:
                print("██", end="")
            elif val > 0.2:
                print("▓▓", end="")
            elif val > 0.05:
                print("░░", end="")
            else:
                print("··", end="")
        print()

## 10. 성능 분석

In [ ]:
import time

# Attention 계산 복잡도 분석
print("⚡ Attention 계산 복잡도")
print("=" * 50)

# 다양한 시퀀스 길이에서 테스트
seq_lengths = [10, 50, 100, 200]
d_model = 64

print(f"d_model = {d_model}\n")
print("Seq Length | Time (ms) | Memory (MB) | Complexity")
print("-" * 55)

for seq_len in seq_lengths:
    # 입력 생성
    Q = K = V = np.random.randn(seq_len, d_model)
    
    # 시간 측정
    start = time.time()
    for _ in range(100):
        _, _ = scaled_dot_product_attention(Q, K, V)
    elapsed = (time.time() - start) / 100 * 1000  # ms
    
    # 메모리 추정 (attention matrix)
    memory_mb = (seq_len * seq_len * 8) / (1024 * 1024)  # 8 bytes per float64
    
    print(f"{seq_len:10d} | {elapsed:9.2f} | {memory_mb:11.2f} | O(n²)")

print("\n💡 관찰:")
print("  - 시퀀스 길이의 제곱에 비례하는 계산량")
print("  - 긴 시퀀스에서 메모리 문제 발생 가능")
print("  - 이래서 효율적인 Attention 변형들이 연구됨")

## 🎉 축하합니다!

Attention 메커니즘의 핵심을 모두 구현하고 이해했습니다!

### 배운 내용:
1. ✅ Scaled Dot-Product Attention
2. ✅ Self-Attention과 Cross-Attention
3. ✅ Causal Masking
4. ✅ Multi-Head Attention
5. ✅ Positional Encoding
6. ✅ 간단한 Transformer Block

### 다음 단계:
- Day 4: 완전한 Transformer 구현
- Layer Normalization, Residual Connection
- Encoder-Decoder 구조

"Attention is literally all you need!"